In [2]:
import pprint
import time

import zmq
from sympy.physics.units import current

ctx = zmq.Context()
# The REQ talks to Pupil remote and receives the session unique IPC SUB PORT
socket = ctx.socket(zmq.REQ)

ip = 'localhost'
port = 50020

socket.connect(f'tcp://{ip}:{port}')

# Request 'SUB_PORT' for reading data
socket.send_string('SUB_PORT')
sub_port = socket.recv_string()

# Request 'PUB_PORT' for writing data
socket.send_string('PUB_PORT')
pub_port = socket.recv_string()

socket.close()

In [3]:
def create_socket(ctx_c, ip_c, port_sub, topics):
    sub = ctx_c.socket(zmq.SUB)
    sub.connect(f'tcp://{ip_c}:{port_sub}')
    for topic in topics:
        sub.subscribe(topic)
    return sub

In [7]:
import msgpack
import numpy as np
from Examples.src.MovementTracker import MovementTracker

# Receive real-time gaze data
gaze_socket = create_socket(ctx, 'localhost', sub_port, ['gaze.'])
fixation_socket = create_socket(ctx, 'localhost', sub_port, ['fixation'])

topic, payload = fixation_socket.recv_multipart()
origin = msgpack.loads(payload)
origin = origin.get('norm_pos', None)
print(f"Fixation data: {origin}")

# compute the distance between the origin and the center of the screen (0.5, 0.5)
distance = np.linalg.norm(np.array(origin) - np.array([0.5, 0.5]))
if distance > 0.1:
    print("Lamp Off")

tracker = MovementTracker(origin, threshold=100, min_dist=0.01, max_dist=0.2, epsilon=0.1)

# Real-time processing loop
while True:
    norm_pos = []
    for i in range(5):
        topic, payload = gaze_socket.recv_multipart()
        message = msgpack.loads(payload)
        norm_pos.append(message.get('norm_pos', None))
    norm_pos = np.array(norm_pos).mean(axis=0)
    result = tracker.update(norm_pos)
    if result is None:
        break
    if result and result['returned']:
        print(result)
        break


Fixation data: [0.4812265428191148, 0.32904399426611924]
Lamp Off
{'moved': True, 'direction': 'right', 'returned': True}


In [ ]:
# Receive real-time gaze data
gaze_socket = create_socket(ctx, 'localhost', sub_port, ['gaze.'])
fixation_socket = create_socket(ctx, 'localhost', sub_port, ['fixation'])

topic, payload = fixation_socket.recv_multipart()
origin = msgpack.loads(payload)
origin = origin.get('norm_pos', None)
print(f"Fixation data: {origin}")

# compute the distance between the origin and the center of the screen (0.5, 0.5)
distance = np.linalg.norm(np.array(origin) - np.array([0.5, 0.5]))
print(f"Distance: {distance}")
if distance > 0.35:
    print("Lamp Off")